# Lecture 12

In [1]:
# symbolic computation tools
import sympy as sp
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi
from sympy import Matrix, simplify, Function, diff, Derivative, nsimplify

import numpy as np

from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()


# styling for plots
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
    


Define our symbolic variables that define our robot configuration

In [2]:
# We wrap in parentheses here so we can write it on multiple lines. Similar
# with the triple quotes on the string. Usually we don't need to use these things.
(t, 
 theta_1, 
 theta_2, 
 theta_3, 
 l_1, 
 l_2, 
 l_3) = symbols("""t, 
                         theta_1 
                         theta_2 
                         theta_3 
                         l_1 
                         l_2 
                         l_3 """ , real = True)


theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

Let's define our homogeneous transformation matrix that applies a rotation and a translation to vectors and matrices

In [3]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

In [4]:
def draw_robot(ordered_list_of_transformations):
    """
    Draw the configuration of the three-link jumper.
    """
    ax_lw = 3
    link_lw = 1

    # homogeneous unit vectors 
    x = np.array([[1, 0, 1]]).T 
    y = np.array([[0, 1, 1]]).T
    origin = np.array([[0, 0, 1]]).T

    prev_origin = np.array([[0, 0, 1]]).T    
    current_transformation = np.eye(3)

    plt.clf()
    ax = plt.gca()  # get current axis

    # now we plot
    plt.plot([prev_origin[0][0]], [prev_origin[1][0]], '-ko',linewidth=link_lw)
    plt.plot([prev_origin[0][0], x[0][0]], [prev_origin[1][0], x[1][0]], '-ro', linewidth=ax_lw)
    plt.plot([prev_origin[0][0], y[0][0]], [prev_origin[1][0], y[1][0]], '-go', linewidth=ax_lw)


    # loop the transforms in order of T_01, T_12, ... , T_N-1N
    for k, transform in enumerate(ordered_list_of_transformations):

        # update the transformation
        current_transformation = current_transformation @ transform
        new_origin = current_transformation @ origin
        new_x = current_transformation @ x
        new_y = current_transformation @ y

        # now we plot
        plt.plot([prev_origin[0][0], new_origin[0][0]], 
                 [prev_origin[1][0], new_origin[1][0]], 
                 '-ko',linewidth=link_lw)
        
        plt.plot([new_origin[0][0], new_x[0][0]], [new_origin[1][0], new_x[1][0]], '-r', linewidth=ax_lw)
        plt.plot([new_origin[0][0], new_y[0][0]], [new_origin[1][0], new_y[1][0]], '-g', linewidth=ax_lw)


#         ax.annotate('{'+str(k+1)+'}', xy=new_origin[0:2][0], xytext=new_origin[0:2][0])

        # lastly update the current_origin to the new_origin
        prev_origin = new_origin

    # lastly set the axes to be equal and appropriate
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')    

## Derive mass matrix from Jacobian and compare with E-L derived version

We will construct the Jacobians the FK for each point mass at a joint, then the full Jacobian for all point masses, then the mass matrix



Define the coordinate systems

![](kinematics.png)

Define helper transforms

In [5]:
T_01 = T(theta_1, 0, 0)
T_12 = T(theta_2, l_1, 0)
T_23 = T(theta_3, l_2, 0)


## Let's start with a two-link

In [6]:
FK1 = (T_01 * Matrix([[l_1], [0], [1]]))[:-1,0]
FK1

Matrix([
[l_1*cos(theta_1(t))],
[l_1*sin(theta_1(t))]])

In [7]:
FK2 = simplify((T_01 * T_12 * Matrix([[l_2], [0], [1]]))[:-1,0])
FK2

Matrix([
[l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t))],
[l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t))]])

In [8]:
J1 = FK1.jacobian([theta_1, theta_2])
J1

Matrix([
[-l_1*sin(theta_1(t)), 0],
[ l_1*cos(theta_1(t)), 0]])

In [9]:
J2 = simplify(FK2.jacobian([theta_1, theta_2]))
J2

Matrix([
[-l_1*sin(theta_1(t)) - l_2*sin(theta_1(t) + theta_2(t)), -l_2*sin(theta_1(t) + theta_2(t))],
[ l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)),  l_2*cos(theta_1(t) + theta_2(t))]])

## Make the mass matrix in cartesian coords

In [10]:
(m_1, 
 m_2, 
 m_3) = symbols("""m_1 
                   m_2 
                   m_3 
                   """ , real = True)


mass_xy = Matrix([[m_1, 0, 0, 0],
                  [0, m_1, 0, 0],
                  [0, 0, m_2, 0],
                  [0, 0, 0, m_2 ]])
mass_xy

Matrix([
[m_1,   0,   0,   0],
[  0, m_1,   0,   0],
[  0,   0, m_2,   0],
[  0,   0,   0, m_2]])

In [11]:
J = Matrix([[J1], [J2]])
J

Matrix([
[                                   -l_1*sin(theta_1(t)),                                 0],
[                                    l_1*cos(theta_1(t)),                                 0],
[-l_1*sin(theta_1(t)) - l_2*sin(theta_1(t) + theta_2(t)), -l_2*sin(theta_1(t) + theta_2(t))],
[ l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)),  l_2*cos(theta_1(t) + theta_2(t))]])

In [12]:
M_jacobian = simplify(J.T * mass_xy * J)
M_jacobian

Matrix([
[l_1**2*m_1 + l_1**2*m_2 + 2*l_1*l_2*m_2*cos(theta_2(t)) + l_2**2*m_2, l_2*m_2*(l_1*cos(theta_2(t)) + l_2)],
[                                 l_2*m_2*(l_1*cos(theta_2(t)) + l_2),                          l_2**2*m_2]])

## Use Euler-Lagrange to derive mass matrix

In [13]:
v1 = diff(FK1,t)
KE1 = (1/2)*m_1*(v1.T * v1)[0]
KE1

0.5*m_1*(l_1**2*sin(theta_1(t))**2*Derivative(theta_1(t), t)**2 + l_1**2*cos(theta_1(t))**2*Derivative(theta_1(t), t)**2)

In [14]:
v2 = diff(FK2,t)
KE2 = (1/2)*m_2*(v2.T * v2)[0]
KE2

0.5*m_2*((-l_1*sin(theta_1(t))*Derivative(theta_1(t), t) - l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_1(t) + theta_2(t)))**2 + (l_1*cos(theta_1(t))*Derivative(theta_1(t), t) + l_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_1(t) + theta_2(t)))**2)

# Assemble Lagrangian

In [15]:
T = KE1 + KE2
V = 0

L = T - V
L = simplify(L)
L

0.5*l_1**2*m_1*Derivative(theta_1(t), t)**2 + 0.5*m_2*(l_1**2*Derivative(theta_1(t), t)**2 + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), t)**2 + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + l_2**2*Derivative(theta_1(t), t)**2 + 2*l_2**2*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + l_2**2*Derivative(theta_2(t), t)**2)

# Derive EOMs from Euler-Lagrange equations

In [16]:
EOM_theta_1 = diff(diff(L, Derivative(theta_1, t)), t) - diff(L, theta_1)
EOM_theta_1 = nsimplify(EOM_theta_1)
EOM_theta_1

l_1**2*m_1*Derivative(theta_1(t), (t, 2)) + m_2*(2*l_1**2*Derivative(theta_1(t), (t, 2)) - 4*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - 2*l_1*l_2*sin(theta_2(t))*Derivative(theta_2(t), t)**2 + 4*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), (t, 2)) + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_2(t), (t, 2)) + 2*l_2**2*Derivative(theta_1(t), (t, 2)) + 2*l_2**2*Derivative(theta_2(t), (t, 2)))/2

In [17]:
EOM_theta_2 = diff(diff(L, Derivative(theta_2, t)), t) - diff(L, theta_2)
EOM_theta_2 = nsimplify(EOM_theta_2)
EOM_theta_2

-m_2*(-2*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)**2 - 2*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t))/2 + m_2*(-2*l_1*l_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + 2*l_1*l_2*cos(theta_2(t))*Derivative(theta_1(t), (t, 2)) + 2*l_2**2*Derivative(theta_1(t), (t, 2)) + 2*l_2**2*Derivative(theta_2(t), (t, 2)))/2

# Assemble the mass matrix

In [18]:
mass = sp.symarray('',(2,2))

mass[0,0] = EOM_theta_1.expand().coeff(Derivative(theta_1,t,t))

mass[0,1] = EOM_theta_1.expand().coeff(Derivative(theta_2,t,t))
mass[1,0] = mass[0,1]

mass[1,1] = EOM_theta_2.expand().coeff(Derivative(theta_2,t,t))

mass = Matrix(mass)

In [19]:
mass = simplify(mass)
mass

Matrix([
[l_1**2*m_1 + l_1**2*m_2 + 2*l_1*l_2*m_2*cos(theta_2(t)) + l_2**2*m_2, l_2*m_2*(l_1*cos(theta_2(t)) + l_2)],
[                                 l_2*m_2*(l_1*cos(theta_2(t)) + l_2),                          l_2**2*m_2]])

In [20]:
M_jacobian == mass

True

## What about the Gravity vector in joint torques?

In [21]:
g = symbols("g")

FG_1 = J1.T * Matrix([[0], [-g]])
FG_1

Matrix([
[-g*l_1*cos(theta_1(t))],
[                     0]])

In [22]:
FG_2 = J2.T * Matrix([[0], [-g]])
FG_2

Matrix([
[-g*(l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)))],
[                        -g*l_2*cos(theta_1(t) + theta_2(t))]])

In [23]:
G = FG_1 + FG_2
G

Matrix([
[-g*l_1*cos(theta_1(t)) - g*(l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)))],
[                                                -g*l_2*cos(theta_1(t) + theta_2(t))]])

In [24]:
J1.T

Matrix([
[-l_1*sin(theta_1(t)), l_1*cos(theta_1(t))],
[                   0,                   0]])

## Try the three link now

In [25]:
J1 = FK1.jacobian([theta_1, theta_2, theta_3])
J1

Matrix([
[-l_1*sin(theta_1(t)), 0, 0],
[ l_1*cos(theta_1(t)), 0, 0]])

In [26]:
J2 = simplify(FK2.jacobian([theta_1, theta_2, theta_3]))
J2

Matrix([
[-l_1*sin(theta_1(t)) - l_2*sin(theta_1(t) + theta_2(t)), -l_2*sin(theta_1(t) + theta_2(t)), 0],
[ l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)),  l_2*cos(theta_1(t) + theta_2(t)), 0]])

In [27]:
J3 = simplify(FK3.jacobian([theta_1, theta_2, theta_3]))
J3

NameError: name 'FK3' is not defined

## Make the mass matrix in cartesian coords

In [ ]:
(m_1, 
 m_2, 
 m_3) = symbols("""m_1 
                   m_2 
                   m_3 
                   """ , real = True)


mass_xy = Matrix([[m_1, 0, 0, 0, 0, 0],
                  [0, m_1, 0, 0, 0, 0],
                  [0, 0, m_2, 0, 0, 0],
                  [0, 0, 0, m_2, 0, 0],
                  [0, 0, 0, 0, m_3, 0],
                  [0, 0, 0, 0, 0, m_3]])
mass_xy

In [ ]:
J = Matrix([[J1], [J2], [J3]])
J

In [ ]:
simplify(J.T * mass_xy * J)